In [72]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicator_collection import IndicatorCollection
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
from pandas.io.json import json_normalize
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
trading_account = alpaca_api.get_account()
elevations = json.dumps(trading_account.__dict__)
data = json.loads(elevations)
# json_normalize(data)

In [71]:
# Returns an Earnings entity if symbol is string, or a dict[symbol -> Earnings] if symbol is a list of string.Returns an Earnings entity if symbol is string, or a dict[symbol -> Earnings] if symbol is a list of string.
earnings = alpaca_api.polygon.earnings('GE')
elevations = json.dumps([ob.__dict__ for ob in earnings])
data = json.loads(elevations)
json_normalize(data)

,_raw.symbol,_raw.EPSReportDate,_raw.fiscalPeriod,_raw.fiscalEndDate,_raw.actualEPS,_raw.consensusEPS,_raw.estimatedEPS,_raw.announceTime,_raw.numberOfEstimates,_raw.EPSSurpriseDollar
0,GE,10/30/2018,Q3 2018,09/30/2018,0.14,0.21,0.21,BTO,5,-0.07
1,GE,07/20/2018,Q2 2018,06/30/2018,0.19,0.18,0.18,BTO,5,0.01
2,GE,04/20/2018,Q1 2018,03/31/2018,0.16,0.11,0.11,BTO,5,0.05
3,GE,01/24/2018,Q4 2017,12/31/2017,0.27,0.28,0.28,BTO,6,-0.01
4,GE,10/20/2017,Q3 2017,09/30/2017,0.29,0.49,0.49,BTO,8,-0.20
5,GE,07/21/2017,Q2 2017,06/30/2017,0.28,0.25,0.25,BTO,8,0.03


# Yahoo Market Earnings Data

In [42]:
from src.yahoo_market_event_scraper import YahooEarningsCalendar
import datetime
NY = 'America/New_York'
yec = YahooEarningsCalendar()
now = pd.Timestamp.now(tz=NY)

if now.dayofweek == 4:
    todate = now + datetime.timedelta(days=3)
else:
    todate = now + datetime.timedelta(days=1)
date_from = datetime.datetime(
    now.year, now.month, now.day, 0, 0)
date_to = datetime.datetime(
    todate.year, todate.month, todate.day, 23, 59)

# now = datetime.now()
timestamp_now = datetime.datetime.timestamp(now)
timestamp_now = round(int(timestamp_now),-2)
print(timestamp_now)
print(datetime.datetime.fromtimestamp(timestamp_now))
timestamp_earnings = yec.get_next_earnings_date('XXII')
print(timestamp_earnings)
print(timestamp_earnings/timestamp_now)
print(timestamp_now/timestamp_earnings)
dt_object = datetime.datetime.fromtimestamp(timestamp_earnings)
print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

1574632100
2019-11-24 15:48:20
1573171200
0.9990722277286231
1.0009286338320966
dt_object = 2019-11-07 18:00:00
type(dt_object) = <class 'datetime.datetime'>


In [53]:
import json
from pandas.io.json import json_normalize
timestamp_earnings = yec.earnings_between(date_from, date_to)

elevations = json.dumps(timestamp_earnings)
data = json.loads(elevations)

json_normalize(data)

,ticker,companyshortname,startdatetime,startdatetimetype,epsestimate,epsactual,epssurprisepct,gmtOffsetMilliSeconds,quoteType
0,NAT,Nordic American Tanker Ltd,2019-11-25T18:30:00.000Z,BMO,-0.08,None,None,0,EQUITY
1,A,Agilent Technologies Inc,2019-11-26T02:00:00.000Z,AMC,0.85,None,None,0,EQUITY
2,PANW,Palo Alto Networks Inc,2019-11-26T02:00:00.000Z,AMC,1.03,None,None,0,EQUITY
3,ARWR,Arrowhead Pharmaceuticals Inc,2019-11-26T02:00:00.000Z,AMC,0.15,None,None,0,EQUITY
4,AMBA,Ambarella Inc,2019-11-26T02:00:00.000Z,AMC,0.20,None,None,0,EQUITY
5,ITRN,Ituran Location and Control Ltd,2019-11-25T11:30:00.000Z,BMO,0.44,None,None,0,EQUITY
6,ABIL,Ability Inc,2019-11-25T18:30:00.000Z,BMO,NaN,None,None,0,EQUITY
7,GBDC,Golub Capital BDC Inc,2019-11-26T02:00:00.000Z,AMC,0.32,None,None,0,EQUITY
8,BECN,Beacon Roofing Supply Inc,2019-11-26T02:00:00.000Z,AMC,1.21,None,None,0,EQUITY
9,HPE,Hewlett Packard Enterprise Co,2019-11-26T02:00:00.000Z,AMC,0.46,None,None,0,EQUITY
